In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
import datetime
from sklearn.preprocessing import MinMaxScaler
import csv
from tensorflow.keras.models import load_model

In [14]:
#칼럼들
days = ['일요일','월요일','화요일','수요일','목요일','금요일','토요일']
month = [1,2,3,4,5,6,7,8,9,10,11,12]
train = []
time = ['05~06','06~07','07~08','08~09','09~10','10~11','11~12','12~13','13~14','14~15','15~16','16~17','17~18','18~19','19~20','20~21','21~22','22~23','23~24','24~05']

In [15]:
#역명 CSV 파일을 읽은 후 TRAIN List에 데이터 삽입
f = open('Dataset/train_name.csv','r')
csvReader = csv.reader(f)

for row in csvReader:
    train.append(row)
    
f.close()

In [16]:
data = pd.read_csv('Dataset/All_Data.csv',encoding='CP949')
data.head()

,날짜,월,요일,역명,05~06,06~07,07~08,08~09,09~10,10~11,11~12,12~13,13~14,14~15,15~16,16~17,17~18,18~19,19~20,20~21,21~22,22~23,23~24,24~05
0,2012-01-01,1,일요일,가락시장(340),27,113,137,240,299,337,332,402,444,558,498,531,498,485,359,329,318,303,179,3
1,2012-01-01,1,일요일,가락시장(8),61,120,118,233,281,263,258,308,395,414,370,498,400,336,246,237,281,264,144,56
2,2012-01-01,1,일요일,가산디지털단지(7),81,218,244,415,795,705,859,1038,1303,1567,1849,2096,1926,1803,1273,1092,1079,633,243,18
3,2012-01-01,1,일요일,강남(222),1314,1683,1462,1604,3331,2609,3931,3712,5907,5235,6884,6081,7099,6353,5188,5067,5427,4508,2285,2
4,2012-01-01,1,일요일,강남구청,200,395,280,439,802,986,730,641,1029,1001,851,849,949,752,643,775,945,614,285,9


In [17]:
data_value = (data['역명'] == '강남(222)') & (data['요일'] == '목요일') & (data['월'] == 5)
data_result = data[data_value]
data_result

,날짜,월,요일,역명,05~06,06~07,07~08,08~09,09~10,10~11,11~12,12~13,13~14,14~15,15~16,16~17,17~18,18~19,19~20,20~21,21~22,22~23,23~24,24~05
32721,2012-05-03,5,목요일,강남(222),1057,4540,13659,22466,17041,8481,8458,8278,10081,5651,9338,11555,14937,24093,20037,12689,14091,14583,7039,1920
34583,2012-05-10,5,목요일,강남(222),1079,4319,13459,22395,17690,8308,8211,8027,10143,9544,10123,11218,14970,24752,20020,12202,13565,14997,6952,1998
36445,2012-05-17,5,목요일,강남(222),1118,4315,13460,22643,16768,8467,8179,8468,10232,9029,9766,10623,14979,24786,19353,11958,13775,14670,6705,2047
38307,2012-05-24,5,목요일,강남(222),1073,4225,13012,22417,16886,8747,8026,8133,10135,9470,10286,10978,15302,24637,19086,12225,13591,14400,6852,1993
40169,2012-05-31,5,목요일,강남(222),1099,3645,12474,21014,15545,8252,7482,7438,9891,8759,9899,10724,14487,22987,18543,11961,12684,14162,7036,1890
131231,2013-05-02,5,목요일,강남(222),987,4495,13050,22086,15220,9140,8421,8533,11235,10090,10918,11619,14857,25471,20322,13296,14617,15936,7128,1805
133156,2013-05-09,5,목요일,강남(222),1049,4494,13226,22790,17623,9112,8351,8459,11027,9306,10512,11526,15320,25465,19741,12128,14246,15699,7204,1892
135081,2013-05-16,5,목요일,강남(222),1082,4711,13177,21810,17468,9595,8750,9147,11494,9868,11292,12155,16446,27247,23809,15002,14763,16201,10045,3390
137006,2013-05-23,5,목요일,강남(222),1071,4435,13182,22289,17005,9516,8556,8816,11134,9993,10871,11869,15920,25603,19742,12560,13887,14125,7174,1953
138931,2013-05-30,5,목요일,강남(222),1049,4125,12056,22525,15747,8461,8173,8023,9915,9230,10110,11472,14563,25317,19894,12540,13517,15543,7612,1932


In [18]:
#데이터 전처리
scaler = MinMaxScaler(feature_range=(0,1))
#행렬 다시 설정
temp = data_result['08~09'].values.reshape(-1,1)
trade_count = scaler.fit_transform(temp)

C:\Users\kkag2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [19]:
def create_dataset(signal_data, look_back=1):
    x_arr, y_arr = [], []
    for i in range(len(signal_data) - look_back):
        x_arr.append(signal_data[i:(i + look_back),0])
        y_arr.append(signal_data[i + look_back,0])
        
    x_arr = np.array(x_arr)
    x_arr = np.reshape(x_arr, (x_arr.shape[0], x_arr.shape[1],1))
    
    return x_arr,np.array(y_arr)

In [20]:
#데이터 분리
#훈련
train = trade_count[0:int(len(trade_count) * 0.5)]
#검증
val = trade_count[int(len(trade_count) * 0.5) : int (len(trade_count) * 0.75)]
#시험
test = trade_count[int(len(trade_count) * 0.75): -1]

print("훈련 Data \n", train)
print("검증 Data \n", val)
print("시험 Data \n", test)

훈련 Data 
 [[0.98320721]
 [0.97952731]
 [0.99238105]
 [0.98066757]
 [0.90795066]
 [0.96351197]
 [1.        ]
 [0.94920701]
 [0.97403338]
 [0.98626516]
 [0.30947445]]
검증 Data 
 [[0.94925884]
 [0.9921219 ]
 [0.98559138]
 [0.96289002]
 [0.91069763]
 [0.8976884 ]]
시험 Data 
 [[0.92888981]
 [0.92406966]
 [0.        ]
 [0.85710584]
 [0.88182855]]


In [21]:
x_train, y_train = create_dataset(train, 1)
x_val, y_val = create_dataset(val, 1)
x_test, y_test = create_dataset(test,1)

print("x_train : ",x_train, "y_train : ",y_train)
print("x_val : ",x_val, "y_val : ",y_val)
print("x_test : ",x_test, "y_test : ",y_test)

x_train :  [[[0.98320721]]

 [[0.97952731]]

 [[0.99238105]]

 [[0.98066757]]

 [[0.90795066]]

 [[0.96351197]]

 [[1.        ]]

 [[0.94920701]]

 [[0.97403338]]

 [[0.98626516]]] y_train :  [0.97952731 0.99238105 0.98066757 0.90795066 0.96351197 1.
 0.94920701 0.97403338 0.98626516 0.30947445]
x_val :  [[[0.94925884]]

 [[0.9921219 ]]

 [[0.98559138]]

 [[0.96289002]]

 [[0.91069763]]] y_val :  [0.9921219  0.98559138 0.96289002 0.91069763 0.8976884 ]
x_test :  [[[0.92888981]]

 [[0.92406966]]

 [[0.        ]]

 [[0.85710584]]] y_test :  [0.92406966 0.         0.85710584 0.88182855]


In [22]:
#학습 모델 구성
batch_size = 22
model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(1, 1)))

model.add(LSTM(64, return_sequences=False))

#output
model.add(Dense(1, activation='linear'))

#손실
model.compile(loss='mse', optimizer='adam')

model.summary()

model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=batch_size, epochs=20)

model.save('result.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 1, 50)             10400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 39,905
Trainable params: 39,905
Non-trainable params: 0
_________________________________________________________________
Train on 10 samples, validate on 5 samples
Epoch 1/20
10/10 [==============================] - 2s 182ms/sample - loss: 0.8544 - val_loss: 0.8878
Epoch 2/20
10/10 [==============================] - 0s 1ms/sample - loss: 0.8426 - val_loss: 0.8754
Epoch 3/20
10/10 [==============================] - 0s 2ms/sample - loss: 0.8307 - val_loss: 0.8630
Epoch 4/20
10/10 [==============================] - 0s 1m

In [23]:
#모델 사용
predictions = model.predict(x_test, batch_size)  
real_predictions = scaler.inverse_transform(predictions)  # 0~1의 값으로 정규화된 값을 원래의 크기로 되돌린다.  
print("predicted next 5 min trade count", real_predictions[-1])  # 예측한 건수를 출력한다.

predicted next 5 min trade count [6360.385]
